# Social Distancing Detector

## Importing Libraries

In [1]:
import numpy as np
import argparse
import sys
import cv2
from math import pow, sqrt

## Paths to required files

In [2]:
model = "SSD_MobileNet.caffemodel"
videopath = 'pedestrians.mp4'
prototxt = 'SSD_MobileNet_prototxt.txt'
labels = 'class_labels.txt'

In [3]:
confidencethresh  = 0.3

## Labels

In [4]:
labels = [line.strip() for line in open(labels)]

In [9]:
bounding_box_color = np.random.uniform(0, 255, size=(len(labels), 3))

## Loading Model

In [6]:
network = cv2.dnn.readNetFromCaffe(prototxt, model)

In [7]:
if videopath:
    cap = cv2.VideoCapture(videopath)
else:
    cap = cv2.VideoCapture(0)

In [10]:
while cap.isOpened():
        # Capture one frame after another
    ret, frame = cap.read()

    if not ret:
        break

    (h, w) = frame.shape[:2]

    # Resize the frame to suite the model requirements. Resize the frame to 300X300 pixels
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    network.setInput(blob)
    detections = network.forward()

    pos_dict = dict()
    coordinates = dict()

    # Focal length
    F = 615

    for i in range(detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > confidencethresh:

            class_id = int(detections[0, 0, i, 1])

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')

            # Filtering only persons detected in the frame. Class Id of 'person' is 15
            if class_id == 15.00:

                # Draw bounding box for the object
                cv2.rectangle(frame, (startX, startY), (endX, endY), bounding_box_color[class_id], 2)

                label = "{}: {:.2f}%".format(labels[class_id], confidence * 100)
                print("{}".format(label))


                coordinates[i] = (startX, startY, endX, endY)

                # Mid point of bounding box
                x_mid = round((startX+endX)/2,4)
                y_mid = round((startY+endY)/2,4)

                height = round(endY-startY,4)

                # Distance from camera based on triangle similarity
                distance = (165 * F)/height
                print("Distance(cm):{dist}\n".format(dist=distance))

                # Mid-point of bounding boxes (in cm) based on triangle similarity technique
                x_mid_cm = (x_mid * distance) / F
                y_mid_cm = (y_mid * distance) / F
                pos_dict[i] = (x_mid_cm,y_mid_cm,distance)

    # Distance between every object detected in a frame
    close_objects = set()
    for i in pos_dict.keys():
        for j in pos_dict.keys():
            if i < j:
                dist = sqrt(pow(pos_dict[i][0]-pos_dict[j][0],2) + pow(pos_dict[i][1]-pos_dict[j][1],2) + pow(pos_dict[i][2]-pos_dict[j][2],2))

                # Check if distance less than 2 metres or 200 centimetres
                if dist < 200:
                    close_objects.add(i)
                    close_objects.add(j)

    for i in pos_dict.keys():
        if i in close_objects:
            COLOR = (0,0,255)
        else:
            COLOR = (0,255,0)
        (startX, startY, endX, endY) = coordinates[i]

        cv2.rectangle(frame, (startX, startY), (endX, endY), COLOR, 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        # Convert cms to feet
        cv2.putText(frame, 'Depth: {i} ft'.format(i=round(pos_dict[i][2]/30.48,4)), (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR, 2)

    cv2.namedWindow('Frame',cv2.WINDOW_NORMAL)

    # Show frame
    cv2.imshow('Frame', frame)
    cv2.resizeWindow('Frame',800,600)

    key = cv2.waitKey(1) & 0xFF

    # Press `q` to exit
    if key == ord("q"):
        break


person: 42.74%
Distance(cm):421.05809128630705

person: 30.87%
Distance(cm):377.2304832713755

person: 46.64%
Distance(cm):407.5301204819277

person: 43.89%
Distance(cm):428.1645569620253

person: 53.55%
Distance(cm):424.581589958159

person: 32.84%
Distance(cm):429.978813559322

person: 62.13%
Distance(cm):417.5925925925926

person: 58.37%
Distance(cm):387.30916030534354

person: 35.81%
Distance(cm):780.5769230769231

person: 50.30%
Distance(cm):380.0561797752809

person: 44.32%
Distance(cm):414.18367346938777

person: 40.61%
Distance(cm):774.6183206106871

person: 45.89%
Distance(cm):492.59708737864077

person: 39.33%
Distance(cm):774.6183206106871

person: 38.47%
Distance(cm):412.5

person: 34.12%
Distance(cm):382.92452830188677

person: 53.57%
Distance(cm):476.40845070422534

person: 41.19%
Distance(cm):762.9699248120301

person: 58.14%
Distance(cm):465.4816513761468

person: 43.30%
Distance(cm):751.6666666666666

person: 50.56%
Distance(cm):463.35616438356163

person: 41.72%
Dista

person: 49.65%
Distance(cm):455.04484304932737

person: 41.28%
Distance(cm):316.1214953271028

person: 67.70%
Distance(cm):367.6630434782609

person: 31.99%
Distance(cm):502.3514851485148

person: 74.12%
Distance(cm):362.4107142857143

person: 33.38%
Distance(cm):523.0670103092783

person: 81.91%
Distance(cm):362.4107142857143

person: 41.82%
Distance(cm):336.00993377483445

person: 83.81%
Distance(cm):352.34375

person: 83.27%
Distance(cm):353.57142857142856

person: 83.09%
Distance(cm):363.7096774193548

person: 86.19%
Distance(cm):366.33574007220216

person: 83.57%
Distance(cm):357.306338028169

person: 85.13%
Distance(cm):347.5171232876712

person: 31.95%
Distance(cm):507.375

person: 84.59%
Distance(cm):345.1530612244898

person: 55.79%
Distance(cm):504.85074626865674

person: 84.58%
Distance(cm):340.52013422818794

person: 38.96%
Distance(cm):515.1015228426396

person: 31.86%
Distance(cm):338.25

person: 84.89%
Distance(cm):346.3310580204778

person: 48.77%
Distance(cm):502.35148

person: 37.46%
Distance(cm):429.978813559322

person: 36.20%
Distance(cm):361.12099644128114

person: 31.41%
Distance(cm):253.6875

person: 38.24%
Distance(cm):417.5925925925926

person: 37.95%
Distance(cm):250.55555555555554

person: 37.16%
Distance(cm):363.7096774193548

person: 34.79%
Distance(cm):433.65384615384613

person: 64.34%
Distance(cm):371.7032967032967

person: 32.73%
Distance(cm):397.94117647058823

person: 66.26%
Distance(cm):384.375

person: 51.08%
Distance(cm):422.8125

person: 57.94%
Distance(cm):435.51502145922746

person: 50.17%
Distance(cm):409.1733870967742

person: 60.47%
Distance(cm):426.3655462184874

person: 67.31%
Distance(cm):412.5

person: 30.57%
Distance(cm):431.8085106382979

person: 54.76%
Distance(cm):414.18367346938777

person: 45.59%
Distance(cm):410.82995951417

person: 54.75%
Distance(cm):402.67857142857144

person: 37.59%
Distance(cm):421.05809128630705

person: 35.45%
Distance(cm):459.1628959276018

person: 31.18%
Distance(cm):424.581589958159

pe

person: 39.43%
Distance(cm):467.62672811059906

person: 36.95%
Distance(cm):417.5925925925926

person: 63.41%
Distance(cm):520.3846153846154

person: 45.49%
Distance(cm):394.84435797665367

person: 45.18%
Distance(cm):449.00442477876106

person: 31.55%
Distance(cm):825.0

person: 54.26%
Distance(cm):504.85074626865674

person: 43.25%
Distance(cm):443.12227074235807

person: 40.66%
Distance(cm):412.5

person: 31.97%
Distance(cm):825.0

person: 47.24%
Distance(cm):512.5

person: 34.08%
Distance(cm):852.7310924369748

person: 32.24%
Distance(cm):439.2857142857143

person: 47.91%
Distance(cm):507.375

person: 35.27%
Distance(cm):431.8085106382979

person: 34.81%
Distance(cm):421.05809128630705

person: 30.80%
Distance(cm):838.6363636363636

person: 60.18%
Distance(cm):504.85074626865674

person: 50.70%
Distance(cm):409.1733870967742

person: 43.60%
Distance(cm):431.8085106382979

person: 32.43%
Distance(cm):838.6363636363636

person: 53.32%
Distance(cm):525.7772020725389

person: 47.70%
Di

person: 64.93%
Distance(cm):465.4816513761468

person: 53.41%
Distance(cm):852.7310924369748

person: 33.08%
Distance(cm):551.4945652173913

person: 32.33%
Distance(cm):593.421052631579

person: 63.21%
Distance(cm):487.8605769230769

person: 51.76%
Distance(cm):348.7113402061856

person: 46.35%
Distance(cm):845.625

person: 36.35%
Distance(cm):566.8994413407821

person: 32.31%
Distance(cm):607.6347305389221

person: 31.48%
Distance(cm):611.2951807228916

person: 56.50%
Distance(cm):451.0

person: 46.50%
Distance(cm):831.7622950819672

person: 36.43%
Distance(cm):370.34671532846716

person: 33.02%
Distance(cm):607.6347305389221

person: 31.46%
Distance(cm):607.6347305389221

person: 30.50%
Distance(cm):638.2075471698113

person: 62.97%
Distance(cm):441.19565217391306

person: 41.52%
Distance(cm):811.8

person: 31.36%
Distance(cm):604.0178571428571

person: 30.86%
Distance(cm):576.5625

person: 65.80%
Distance(cm):457.0945945945946

person: 54.47%
Distance(cm):667.5986842105264

person: 

person: 60.11%
Distance(cm):465.4816513761468

person: 52.23%
Distance(cm):499.8768472906404

person: 38.84%
Distance(cm):512.5

person: 38.80%
Distance(cm):622.5460122699386

person: 34.78%
Distance(cm):534.078947368421

person: 33.61%
Distance(cm):650.4807692307693

person: 31.72%
Distance(cm):593.421052631579

person: 84.51%
Distance(cm):451.0

person: 52.21%
Distance(cm):459.1628959276018

person: 51.32%
Distance(cm):520.3846153846154

person: 46.14%
Distance(cm):536.9047619047619

person: 31.84%
Distance(cm):435.51502145922746

person: 30.98%
Distance(cm):509.9246231155779

person: 80.77%
Distance(cm):463.35616438356163

person: 49.59%
Distance(cm):534.078947368421

person: 46.14%
Distance(cm):451.0

person: 35.20%
Distance(cm):463.35616438356163

person: 71.19%
Distance(cm):480.92417061611377

person: 53.14%
Distance(cm):536.9047619047619

person: 51.74%
Distance(cm):447.0264317180617

person: 49.02%
Distance(cm):560.6353591160221

person: 34.77%
Distance(cm):504.85074626865674



person: 82.07%
Distance(cm):507.375

person: 58.52%
Distance(cm):536.9047619047619

person: 55.53%
Distance(cm):512.5

person: 50.19%
Distance(cm):467.62672811059906

person: 44.75%
Distance(cm):542.6470588235294

person: 44.36%
Distance(cm):504.85074626865674

person: 80.44%
Distance(cm):534.078947368421

person: 59.50%
Distance(cm):545.5645161290323

person: 45.41%
Distance(cm):525.7772020725389

person: 43.78%
Distance(cm):492.59708737864077

person: 31.64%
Distance(cm):563.75

person: 30.97%
Distance(cm):304.72972972972974

person: 82.22%
Distance(cm):525.7772020725389

person: 56.66%
Distance(cm):528.515625

person: 52.14%
Distance(cm):499.8768472906404

person: 39.99%
Distance(cm):467.62672811059906

person: 36.08%
Distance(cm):289.1025641025641

person: 86.94%
Distance(cm):507.375

person: 59.11%
Distance(cm):525.7772020725389

person: 56.58%
Distance(cm):487.8605769230769

person: 40.44%
Distance(cm):285.04213483146066

person: 36.43%
Distance(cm):445.0657894736842

person: 84.

person: 50.78%
Distance(cm):474.18224299065423

person: 46.48%
Distance(cm):375.8333333333333

person: 46.25%
Distance(cm):622.5460122699386

person: 35.87%
Distance(cm):404.2828685258964

person: 30.26%
Distance(cm):630.27950310559

person: 54.44%
Distance(cm):573.3050847457627

person: 54.04%
Distance(cm):615.0

person: 48.15%
Distance(cm):495.0

person: 46.56%
Distance(cm):378.6380597014925

person: 32.64%
Distance(cm):622.5460122699386

person: 30.34%
Distance(cm):451.0

person: 30.22%
Distance(cm):407.5301204819277

person: 55.84%
Distance(cm):563.75

person: 51.85%
Distance(cm):478.6556603773585

person: 49.30%
Distance(cm):611.2951807228916

person: 36.87%
Distance(cm):396.38671875

person: 36.52%
Distance(cm):618.75

person: 51.17%
Distance(cm):478.6556603773585

person: 48.78%
Distance(cm):589.9709302325581

person: 39.86%
Distance(cm):563.75

person: 30.17%
Distance(cm):414.18367346938777

person: 58.92%
Distance(cm):583.1896551724138

person: 56.37%
Distance(cm):492.59708737

person: 45.47%
Distance(cm):517.7295918367347

person: 44.71%
Distance(cm):437.39224137931035

person: 40.98%
Distance(cm):375.8333333333333

person: 40.07%
Distance(cm):528.515625

person: 51.75%
Distance(cm):435.51502145922746

person: 40.67%
Distance(cm):439.2857142857143

person: 36.11%
Distance(cm):523.0670103092783

person: 31.60%
Distance(cm):534.078947368421

person: 31.13%
Distance(cm):352.34375

person: 45.90%
Distance(cm):443.12227074235807

person: 36.42%
Distance(cm):435.51502145922746

person: 34.40%
Distance(cm):551.4945652173913

person: 47.62%
Distance(cm):459.1628959276018

person: 38.20%
Distance(cm):542.6470588235294

person: 57.28%
Distance(cm):443.12227074235807

person: 37.07%
Distance(cm):542.6470588235294

person: 50.22%
Distance(cm):437.39224137931035

person: 40.21%
Distance(cm):517.7295918367347

person: 57.40%
Distance(cm):447.0264317180617

person: 36.65%
Distance(cm):520.3846153846154

person: 53.38%
Distance(cm):445.0657894736842

person: 45.70%
Distance

In [11]:
cap.release()
cv2.destroyAllWindows()